In [1]:
import os
import pandas as pd
import numpy as np
import joblib

import tensorflow as tf
from keras.utils import to_categorical
from keras.models import load_model

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report

2025-03-25 22:52:13.107901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-25 22:52:13.242582: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 22:52:13.253863: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-25 22:52:13.253880: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [128]:
!pwd

/home/jovana/code/y01a-BG/NeuroPredict/notebooks


In [5]:
data = pd.read_csv('/home/jovana/code/y01a-BG/NeuroPredict/processed_data/data.csv')

In [6]:
data.shape

(6894, 179)

In [7]:
X = data.drop(columns = 'y')
y = data.y -1
#Merginf Class 1 and 2 together
y = y.replace({2: 1})
print(f"✅ Data separated (X,y- 2 Classes)")

✅ Data separated (X,y- 2 Classes)


In [10]:
y.shape

(6894,)

In [11]:
np.unique(y)

array([0, 1])

In [9]:
X.shape

(6894, 178)

In [2]:
############################################
####### Loading models of interest ########
###########################################

# Load trianed model from model/ .h5 file
lstm = load_model("/home/jovana/code/y01a-BG/NeuroPredict/models/model_lstm.h5")
binary_rocket = joblib.load("/home/jovana/code/y01a-BG/NeuroPredict/models/binary_rocket.pkl")

2025-03-25 11:43:07.367477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-25 11:43:07.367702: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-25 11:43:07.367742: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-03-25 11:43:07.367776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-03-25 11:43:07.367808: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [36]:
####################################################
####### Loading/ LSTM - encoding test data   ########
####################################################

X_test = pd.read_csv("/home/jovana/code/y01a-BG/NeuroPredict/processed_data/X_test.csv")
y_test = pd.read_csv("/home/jovana/code/y01a-BG/NeuroPredict/processed_data/y_test.csv")
y_test = y_test -1 # Shift labels to start from 0
print(f"✅ X_test, y_test loaded")

X_test_lstm = np.expand_dims(X_test, axis=2)  # to be able to enter the LSTM model
y_test_lstm = y_test-1
y_test_lstm = to_categorical(y_test, num_classes=3)

#print(y_test)
print(f"✅ X_test_lstm, y_test_lstm  are ready.")

✅ X_test, y_test loaded
✅ X_test_lstm, y_test_lstm  are ready.


In [37]:
 X_test_lstm.shape

(1380, 178, 1)

In [54]:
# 1st poredictions for LSTM
y_lstm_probs = lstm.predict(X_test_lstm)
y_lstm_preds = np.argmax(y_lstm_probs, axis=1)
print(f"Unique values in y_lstm_preds before conversion: {np.unique(y_lstm_preds)}")
print(f'Classes population after lstm : {pd.DataFrame(y_lstm_preds).value_counts()}')

44/44 [==============================] - 3s 74ms/step
Unique values in y_lstm_preds before conversion: [0 1 2]
Classes population after lstm : 0
1    544
0    463
2    373
Name: count, dtype: int64


In [55]:
# Define the category to exclude
category_to_exclude = 0  # Replace with actual category

# Create a mask for the rows to keep
mask = y_lstm_preds != category_to_exclude 
mask

array([False,  True, False, ..., False,  True, False])

In [43]:
# getting the data for Rocket
X_test_rocket = X_test[mask]
y_test_rocket = y_test[mask]

In [42]:
X_test_rocket.shape

(917, 178)

In [46]:
y_test_rocket.value_counts()

y
2    469
1    437
0     11
Name: count, dtype: int64

In [47]:
y_test_rocket.shape

(917, 1)

In [48]:
# Rocket encoding for X
def from_2d_array_to_nested(X_train: pd.DataFrame, y_train: pd.DataFrame = None, index=None, columns=None, time_index=None, cells_as_numpy=False):
    """Convert 2D dataframe to nested dataframe."""
    if (time_index is not None) and cells_as_numpy:
        raise ValueError(
            "`Time_index` cannot be specified when `return_arrays` is True, "
            "time index can only be set to pandas Series"
        )
    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.to_numpy()

    container = np.array if cells_as_numpy else pd.Series
    n_instances, n_timepoints = X_train.shape

    if time_index is None:
        time_index = np.arange(n_timepoints)
    kwargs = {"index": time_index}

    Xt = pd.DataFrame(
        pd.Series([container(X_train[i, :], **kwargs) for i in range(n_instances)])
    )
    if index is not None:
        Xt.index = index
    if columns is not None:
        Xt.columns = columns
    return Xt

In [50]:
# encoding data for Rocket
X_test_rocket =from_2d_array_to_nested(X_test_rocket, index=None, columns=None, time_index=None, cells_as_numpy=False)

In [127]:
X_test_rocket.shape

(4, 178, 1)

In [51]:
# Step 3: Use ROCKET for the cases where LSTM predicted Class 1 or 2
rocket_preds = binary_rocket.predict(X_test_rocket)  # Predict with ROCKET for Class 1 and 2
rocket_preds

array([2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 3, 2, 2,
       2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2,
       2, 2, 3, 3, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2,
       2, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2,
       2, 3, 2, 2, 2, 3, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 3, 3, 2,
       3, 2, 3, 2, 3, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2, 2,
       2, 3, 3, 2, 2, 3, 2, 2, 3, 2, 2, 3, 3, 3, 3, 2, 3, 2, 2, 2, 2, 3,
       3, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2,
       2, 2, 3, 2, 3, 2, 3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 3,
       2, 3, 2, 3, 2, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 3, 2,
       2, 2, 3, 3, 2, 3, 2, 3, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 2, 3, 2, 3,
       3, 2, 2, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 2, 3, 2, 3,
       3, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 3,

In [52]:
pd.DataFrame(rocket_preds).value_counts()

0
2    541
3    376
Name: count, dtype: int64

In [61]:
# Step 4: Replace LSTM predictions with ROCKET predictions for Class 1 & 2 cases
y_lstm_preds_mask = y_lstm_preds[mask] 
y_lstm_preds_mask.shape

(917,)

In [ ]:
y_lstm_preds[mask] = rocket_preds

In [62]:
rocket_preds.shape

(917,)

In [63]:
y_lstm_preds

array([0, 1, 0, ..., 0, 2, 0])

In [73]:
####################################################
####### Evaluating the hybrid model  ########
####################################################

def hybrid_prediction(X_test_lstm, lstm, binary_rocket):
    """
    X_test_lstm -> input data for LSTM
    X_test_rocket -> input data for ROCKET (only Class 1 & 2)
    lstm -> Trained LSTM model
    binary_rocket -> Trained ROCKET model
    """
    # Step 1: Get LSTM predictions
    y_lstm_probs = lstm.predict(X_test_lstm)  # Assume this gives probabilities
    y_lstm_preds = np.argmax(y_lstm_probs, axis=1)  # Predicted classes from LSTM

    # Debug
    print(f"Unique values in y_lstm_preds before conversion: {np.unique(y_lstm_preds)}")
    print(f'Classes population after lstm : {pd.DataFrame(y_lstm_preds).value_counts()}')

    # Define the category to exclude
    category_to_exclude = 0  # Replace with actual category
    # Create a mask for the rows to keep 
    mask = y_lstm_preds != category_to_exclude 
    # With maskcreate X_test_rocket, y_test_rocket
    X_test_rocket = X_test[mask]
    y_test_rocket = y_test[mask]
    # encoding data for Rocket
    X_test_rocket =from_2d_array_to_nested(X_test_rocket, index=None, columns=None, time_index=None, cells_as_numpy=False)

    rocket_preds = binary_rocket.predict(X_test_rocket)  # Predict with ROCKET for Class 1 and 2

    # Step 4: Replace LSTM predictions with ROCKET predictions for Class 1 & 2 cases
    y_lstm_preds[mask] = rocket_preds

    # Step 5: Ensure all predictions are within the valid range of classes [0, 1, 2]
    y_lstm_preds = np.clip(y_lstm_preds, 0, 2)  # Ensure the class predictions are within [0, 2]

    return y_lstm_preds



In [74]:
# Example usage:
# Assuming `lstm` and `binary_rocket` are already trained models
y_lstm_preds = hybrid_prediction(X_test_lstm,lstm, binary_rocket)

44/44 [==============================] - 3s 75ms/step
Unique values in y_lstm_preds before conversion: [0 1 2]
Classes population after lstm : 0
1    544
0    463
2    373
Name: count, dtype: int64


In [85]:
y_lstm_preds

array([0, 2, 0, ..., 0, 2, 0])

In [83]:
y_test.value_counts()

y
2    474
1    457
0    449
Name: count, dtype: int64

In [86]:
y_test

,y
0,0
1,1
2,0
3,1
4,2
...,...
1375,1
1376,0
1377,0
1378,2


In [87]:

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_lstm_preds)

# Calculate precision, recall, and f1-score (macro average)
precision = precision_score(y_test, y_lstm_preds, average='macro')  # 'macro' averages metrics across all classes
recall = recall_score(y_test, y_lstm_preds, average='macro')
f1 = f1_score(y_test, y_lstm_preds, average='macro')

# Display results as a table
results = {
        'Metric': ['Accuracy', 'Precision (Macro)', 'Recall (Macro)', 'F1-Score (Macro)'],
        'Score': [accuracy, precision, recall, f1]
}

# Create DataFrame for easy display
results_df = pd.DataFrame(results)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_lstm_preds)
print("✅ \nConfusion Matrix:")
print(conf_matrix)

# Create classification report (including precision, recall, f1-score per class)
class_report = classification_report(y_test, y_lstm_preds, target_names=['Class 0', 'Class 1', 'Class 2'])
print("✅ \nClassification Report:")
print(class_report)

print(f"✅ Metrics Table:\n{results_df}")





✅ 
Confusion Matrix:
[[438   0  11]
 [ 20   0 437]
 [  5   0 469]]
✅ 
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.95      0.98      0.96       449
     Class 1       0.00      0.00      0.00       457
     Class 2       0.51      0.99      0.67       474

    accuracy                           0.66      1380
   macro avg       0.49      0.65      0.54      1380
weighted avg       0.48      0.66      0.54      1380

✅ Metrics Table:
              Metric     Score
0           Accuracy  0.657246
1  Precision (Macro)  0.485818
2     Recall (Macro)  0.654984
3   F1-Score (Macro)  0.544954


/home/jovana/.pyenv/versions/3.10.6/envs/NeuroPredict/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jovana/.pyenv/versions/3.10.6/envs/NeuroPredict/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jovana/.pyenv/versions/3.10.6/envs/NeuroPredict/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

In [105]:
# Step 1: Get LSTM predictions
y_lstm_probs = lstm.predict(X_pred)  # Assuming LSTM outputs probability scores
y_lstm_preds = np.argmax(y_lstm_probs, axis=1) 

1/1 [==============================] - 0s 89ms/step


In [106]:
y_lstm_preds

array([0, 0, 2, 1, 1, 2])

In [107]:
# Define the category to exclude
category_to_exclude = 0  # Replace with actual category
mask = y_lstm_preds != category_to_exclude 
    

In [108]:
mask

array([False, False,  True,  True,  True,  True])

In [122]:
# With maskcreate X_test_rocket, y_test_rocket
X_pred_rocket = X_pred[mask]
X_pred_rocket.shape

(4, 178, 1)

In [124]:
from sktime.datatypes._panel._convert import from_2d_array_to_nested
X_pred_rocket = X_pred_rocket.squeeze(-1)  # Removes the last dimension
print(X_pred_rocket.shape)  # Should now be (4, 178)

# Convert to nested format
X_pred_rocket = from_2d_array_to_nested(X_pred_rocket)

(4, 178)


In [125]:
 rocket_preds = binary_rocket.predict(X_pred_rocket)

In [126]:
 rocket_preds

array([2, 2, 2, 2])

In [100]:
def hybrid_prediction_pred(X_pred, lstm, binary_rocket):
    """
    X_pred -> New input data
    lstm -> Trained LSTM model
    binary_rocket -> Trained ROCKET model
    Returns final hybrid predictions for X_pred
    """

    # Step 1: Get LSTM predictions
    y_lstm_probs = lstm.predict(X_pred)  # Assuming LSTM outputs probability scores
    y_lstm_preds = np.argmax(y_lstm_probs, axis=1)  # Convert to class labels

    # Debugging
    print(f"Unique values in y_lstm_preds before conversion: {np.unique(y_lstm_preds)}")
    print(f'Classes population after lstm : {pd.DataFrame(y_lstm_preds).value_counts()}')

    # Define the category to exclude
    category_to_exclude = 0  # Replace with actual category
    # Create a mask for the rows to keep 
    mask = y_lstm_preds != category_to_exclude 
    # With maskcreate X_test_rocket, y_test_rocket
    X_test_rocket = X_pred[mask]
    # encoding data for Rocket
    X_test_rocket =from_2d_array_to_nested(X_test_rocket, index=None, columns=None, time_index=None, cells_as_numpy=False)

#     # Step 3: Convert to nested format for ROCKET
#     if X_pred_rocket.shape[0] > 0:  # Avoid empty case
#         X_pred_rocket = from_2d_array_to_nested(X_pred_rocket)

        # Step 4: Predict with ROCKET for Class 1 & 2
        rocket_preds = binary_rocket.predict(X_pred_rocket)

        # Step 5: Replace LSTM predictions with ROCKET predictions for Class 1 & 2
        y_lstm_preds[mask] = rocket_preds

    # Step 6: Ensure valid class predictions [0, 1, 2]
    y_lstm_preds = np.clip(y_lstm_preds, 0, 2)

    return y_lstm_preds

In [101]:
y_pred = hybrid_prediction_pred(X_pred, lstm,binary_rocket)

print("Final predictions:", y_pred)

1/1 [==============================] - 0s 66ms/step
Unique values in y_lstm_preds before conversion: [0 1 2]
Classes population after lstm : 0
0    2
1    2
2    2
Name: count, dtype: int64


ValueError: too many values to unpack (expected 2)